Fine-tuning best BART 🤖
-----------------------------------

In this notebook, we will continue the fine-tuning of the BART model on the new extracted sentences from the book **Grammaire de Wolof Moderne** and additional sentences. We provide, bellow, the main evaluation figures, obtained from the hyperparameter search step. We will evaluate the training on the validation dataset.

- Parallel coordinates from panel:

- Parameter importance char: 
[t5_v3_importance](https://wandb.ai/oumar-kane-team/small-t5-cross-fw-translation-bayes-hpsearch-v3/reports/undefined-23-05-16-10-36-17---Vmlldzo0Mzc4NDY0?accessToken=eyaiyrid0qz1zg2jkq3fc65biw53084dpfitbi0dgonq6mweupw6kgjml9d2nv1w)

We can see in the above chart that the batch is the most important parameter with a negative correlation with the BLEU score (meaning that a lower batch size is better). Next, we the probability of modifying a character in the french corpus is also important and a high probability provide a better BLEU score.  

In [1]:
# let us import all necessary libraries
from transformers import BartModel, BartForConditionalGeneration, Seq2SeqTrainer, BartTokenizerFast, set_seed, AdamW, get_linear_schedule_with_warmup,\
                          get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup, Adafactor
from wolof_translate.utils.sent_transformers import TransformerSequences
from wolof_translate.utils.improvements.end_marks import add_end_mark # added
from torch.nn import TransformerEncoderLayer, TransformerDecoderLayer
from torch.utils.data import Dataset, DataLoader, random_split
from wolof_translate.data.dataset_v4 import SentenceDataset # v2 -> v3 -> v4
from wolof_translate.utils.sent_corrections import *
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import _LRScheduler
# from custom_rnn.utils.kwargs import Kwargs
from torch.nn.utils.rnn import pad_sequence
from plotly.subplots import make_subplots
from nlpaug.augmenter import char as nac
from torch.utils.data import DataLoader
# from datasets  import load_metric # make pip install evaluate instead
# and pip install sacrebleu for instance
from torch.nn import functional as F
import plotly.graph_objects as go
from tokenizers import Tokenizer
import matplotlib.pyplot as plt
import pytorch_lightning as lt
from tqdm import tqdm, trange
from functools import partial
from torch.nn import utils
from copy import deepcopy
from torch import optim
from typing import *
from torch import nn
import pandas as pd
import numpy as np
import itertools
import evaluate
import random
import string
import shutil
import wandb
import torch
import json
import copy
import os

# add seed for everything
lt.seed_everything(0)

os.environ["WANDB_DISABLED"] = "true"

Global seed set to 0


## French to wolof

### Configure dataset 🔠

In [2]:
# recuperate the tokenizer from a json file
tokenizer = BartTokenizerFast(tokenizer_file=f"wolof-translate/wolof_translate/tokenizers/t5_tokenizers/tokenizer_v3_2.json")

In [3]:
def recuperate_datasets(fr_char_p: float, fr_word_p: float, max_len: int, end_mark_opt: int):

  # Let us recuperate the end_mark adding option
  if end_mark_opt == 1:
    # Create augmentation to add on French sentences
    fr_augmentation_1 = TransformerSequences(nac.KeyboardAug(aug_char_p=fr_char_p, aug_word_p=fr_word_p,
                                                             aug_word_max = max_len),
                                          remove_mark_space, delete_guillemet_space, add_mark_space)

    fr_augmentation_2 = TransformerSequences(remove_mark_space, delete_guillemet_space, add_mark_space)
    
  else:
    
    if end_mark_opt == 2:

      end_mark_fn = partial(add_end_mark, end_mark_to_remove = '!', replace = True)
    
    elif end_mark_opt == 3:

      end_mark_fn = partial(add_end_mark)
    
    elif end_mark_opt == 4:

      end_mark_fn = partial(add_end_mark, end_mark_to_remove = '!')

    # Create augmentation to add on French sentences
    fr_augmentation_1 = TransformerSequences(nac.KeyboardAug(aug_char_p=fr_char_p, aug_word_p=fr_word_p,
                                                             aug_word_max = max_len),
                                          remove_mark_space, delete_guillemet_space, add_mark_space, end_mark_fn)
    
    fr_augmentation_2 = TransformerSequences(remove_mark_space, delete_guillemet_space, add_mark_space, end_mark_fn)
    
  # Recuperate the train dataset
  train_dataset_aug = SentenceDataset(f"data/extractions/new_data/train_set.csv",
                                        tokenizer,
                                        truncation = True, max_len=max_len,
                                        cp1_transformer = fr_augmentation_1,
                                        cp2_transformer = fr_augmentation_2
                                        )

  # Recuperate the valid dataset
  valid_dataset = SentenceDataset(f"data/extractions/new_data/valid_set.csv",
                                        tokenizer, max_len=max_len,
                                        cp1_transformer = fr_augmentation_2,
                                        cp2_transformer = fr_augmentation_2,
                                        truncation = True)
  
  # Return the datasets
  return train_dataset_aug, valid_dataset

### Configure the evaluation function ⚙️

Let us initialize the evaluation object.

In [4]:
%run wolof-translate/wolof_translate/utils/evaluation.py
evaluation = TranslationEvaluation(tokenizer)


### Searching for the best parameters 🕖

In [5]:
from wolof_translate.models.transformers.optimization import TransformerScheduler
from wolof_translate.trainers.transformer_trainer import ModelRunner
# from wolof_translate.utils.evaluation import TranslationEvaluation
from wolof_translate.models.transformers.main import Transformer
from wolof_translate.utils.split_with_valid import split_data


-------------

### --- Wandb v5 2000

In [6]:
# calculate the linear epochs
415*0.1

41.5

In [7]:
# let us initialize the hyperparameter configuration 
config = {
    'random_state': 0,
    'fr_char_p': 0.2663448744764274,
    'fr_word_p': 0.1096679360128458,
    'learning_rate': 0.001334194132566519,
#     'weight_decay': 0.012717464509690656,
    'batch_size': 16,
    'warmup_ratio': 0.0,
    'max_epoch': 415,
    'epochs': 41,
    'mid_epoch': 41,
    'max_len': 72,
    'end_mark': 3,
    'bleu': 4.3749,
    'model_dir': 'fw_bart_custom_train_v5_checkpoints',
    'new_model_dir': 'bart_custom_train_results_fw_v5'
}

# Initialize the model name
model_name = 'facebook/bart-base'

# import the model with its pre-trained weights
model = BartForConditionalGeneration.from_pretrained(model_name,
                                                     eos_token_id = tokenizer.eos_token_id,
                                                     forced_eos_token_id = tokenizer.eos_token_id
                                                     )

# resize the token embeddings
model.resize_token_embeddings(len(tokenizer))

# let us initialize the evaluation class
evaluation = TranslationEvaluation(tokenizer)

# let us initialize the trainer
trainer = ModelRunner(model, seed = 0, version = 5, evaluation = evaluation, optimizer=Adafactor)

# split the data
split_data(config['random_state'], csv_file = "ad_sentences.csv")

# recuperate train and test set
train_dataset, test_dataset = recuperate_datasets(config['fr_char_p'], 
                                                    config['fr_word_p'], config['max_len'],
                                                    config['end_mark'])

# let us calculate the appropriate warmup steps (let us take a max epoch of 100)
length = len(train_dataset)

n_steps = length // config['batch_size']

num_steps = config['max_epoch'] * n_steps

warmup_steps = (config['max_epoch'] * n_steps) * config['warmup_ratio']

# # Initialize the scheduler parameters
scheduler_args = {'num_warmup_steps': warmup_steps, 'num_training_steps': num_steps}

# Initialize the optimizer parameters
optimizer_args = {
    'lr': config['learning_rate'],
#     'weight_decay': config['weight_decay'],
    # 'betas': (0.9, 0.98),
    'warmup_init': False,
    'relative_step': False
}

# Initialize the loaders parameters
train_loader_args = {'batch_size': config['batch_size']}

# Add the datasets and hyperparameters to trainer
trainer.compile(train_dataset, test_dataset, tokenizer, train_loader_args,
                optimizer_kwargs = optimizer_args,
                lr_scheduler=get_linear_schedule_with_warmup,
                lr_scheduler_kwargs=scheduler_args, 
                predict_with_generate = True,
                hugging_face = True,
                logging_dir="data/logs/bart_custom_train_fw"
                )

# We will from checkpoints so let us the model
# trainer.load(config['model_dir'], load_best=True) # Only for the first loading
trainer.load(config['new_model_dir'])

        

### ---

In [ ]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/36 [00:00<?, ?it/s]/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:265: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 6: 
{Learning rate: [0.0013179554776459284]}


Test batch number 13: 100%|██████████| 13/13 [00:16<00:00,  1.28s/batches]



Metrics: {'train_loss': 0.5017463801184086, 'test_loss': 0.6395571048443134, 'bleu': 3.4805, 'gen_len': 10.1869}




  3%|▎         | 1/36 [00:49<28:40, 49.15s/it]

...

Test batch number 13: 100%|██████████| 13/13 [00:14<00:00,  1.15s/batches]



Metrics: {'train_loss': 0.06396503966640342, 'test_loss': 0.8113086384076339, 'bleu': 8.5802, 'gen_len': 12.4949}




 44%|████▍     | 16/36 [09:58<12:14, 36.71s/it]

For epoch 22: 
{Learning rate: [0.0012659917819000377]}


Test batch number 4:  23%|██▎       | 3/13 [00:03<00:11,  1.17s/batches]

### ---

In [ ]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/18 [00:00<?, ?it/s]/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:265: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 24: 
{Learning rate: [0.0012594963199318015]}


Test batch number 13: 100%|██████████| 13/13 [00:24<00:00,  1.90s/batches]



Metrics: {'train_loss': 0.04755422307385339, 'test_loss': 0.846881987956854, 'bleu': 10.4332, 'gen_len': 12.5354}




  6%|▌         | 1/18 [00:57<16:16, 57.44s/it]

...

Test batch number 13: 100%|██████████| 13/13 [00:15<00:00,  1.21s/batches]



Metrics: {'train_loss': 0.04248161056088378, 'test_loss': 0.8589623387043293, 'bleu': 9.0675, 'gen_len': 11.9899}




 11%|█         | 2/18 [01:41<13:10, 49.39s/it]

For epoch 26: 
{Learning rate: [0.0012530008579635652]}


Train batch number 11:  10%|█         | 10/99 [00:03<00:29,  3.01batches/s]

### ---

In [ ]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/15 [00:00<?, ?it/s]/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:265: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 27: 
{Learning rate: [0.001249753126979447]}


Test batch number 13: 100%|██████████| 13/13 [00:28<00:00,  2.16s/batches]



Metrics: {'train_loss': 0.006390525402049675, 'test_loss': 0.8933931864224948, 'bleu': 10.1805, 'gen_len': 12.096}




  7%|▋         | 1/15 [01:12<16:50, 72.15s/it]

...

Test batch number 13: 100%|██████████| 13/13 [00:26<00:00,  2.08s/batches]



Metrics: {'train_loss': 0.030019405674226958, 'test_loss': 0.9254449972739587, 'bleu': 9.0118, 'gen_len': 11.7121}




 53%|█████▎    | 8/15 [10:11<08:59, 77.06s/it]

For epoch 35: 
{Learning rate: [0.0012237712791065018]}


Train batch number 15:  14%|█▍        | 14/99 [00:05<00:36,  2.33batches/s]

### Predictions and Evaluation

In [10]:
# initialize the transformation sequence
end_mark_fn = partial(add_end_mark)
fr_augmentation = TransformerSequences(remove_mark_space, delete_guillemet_space, end_mark_fn)

# let us get the test set
test_dataset = SentenceDataset(f"data/extractions/new_data/test_set.csv",
                                        tokenizer = tokenizer,
                                        cp1_transformer = fr_augmentation,
                                        cp2_transformer = fr_augmentation,
                                        truncation = True)

Let us make the evaluation and print the predicted sentences.

In [9]:
# evaluation with test set
df_ft_to_wf = trainer.evaluate(test_dataset)

Evaluation batch number 13: 100%|██████████| 13/13 [00:24<00:00,  1.92s/batches]

predictions: [[2, 5, 3676, 2863, 153, 1004, 7, 10, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 5, 879, 103, 3388, 56, 1025, 544, 749, 60, 8, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 5, 3333, 1604, 528, 20, 7, 11, 351, 1148, 21, 179, 1752, 72, 138, 74, 20, 810, 8, 55, 39, 17, 409, 175, 57, 482, 19, 9, 60, 39, 17, 175, 57, 791, 8, 177, 747, 14, 191, 17, 60, 8, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 5, 3682, 937, 22, 97, 35, 128, 22, 542, 8, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 5, 3693, 3305, 39, 41, 178, 8, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 


d:\oumar\seagate1\ia data forest\master semestre 3\vision par ordinateur\pytorch_projects\subject2\wolof-translate\wolof_translate\trainers\transformer_trainer.py:649: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  metrics.update(self.evaluation.compute_metrics((np.array(predictions_), np.array(labels_))))


In [12]:
df_ft_to_wf[1].tail(10)

,original_sentences,translations,predictions
188,Donne le travail à un autre!,Joxal kenn liggéey bi!,Deŋk na la ci gën.
189,Cet homme qui a été.,Góor gii demoon.,Gor gii dem.
190,Dites-lui.,Nileen ka.,Kaaye dey.
191,Il est là.,Ma ŋgoogule foofu.,Mi ŋgi foofule.
192,Tu vois cet homme là-bas?,Gis ŋga nit kale?,Gis ŋga nit kee?
193,L'homme est parti je crois!,Ma defe góor gi dem na!,Góor gi dem na ma defe.
194,J'ai aperçu un baobab.,Séen naa ag guy.,Séen naa aw fas.
195,Celui-ci serait parti.,Kii dafa demkoon.,Kii dafa demkoon.
196,Toutes les portes étaient ouvertes.,Bunt yi yépp a tëjju woon.,Seetal sarax su.
197,"C'est Fatim, aujourd'hui.","Faatim la, tay.",Faatim la daal.


In [13]:
# let us display 100 samples
pd.options.display.max_rows = 100
df_ft_to_wf[1].sample(100)

,original_sentences,translations,predictions
18,Afin que parte qui?,Ndax kan dem?,Ndax kan dem?
168,Tu as vu les autres amis?,Gis ŋga sa yeneen xarit yi?,Gis ŋga jigéen ñeneen ñooñu?
63,Quelles femmes se sont égarées?,Jigéen ñan ñoo réer?,Jigéen dañu réer?
175,C'est ce que tu as dit que la jeune femme préf...,Li ŋga wax la ndaw si taamu.,La ŋga wax la.
71,Moi-même je n'ai pas été.,Man mii demuma.,Man demuma.
86,Ils partent.,Dem nañu.,Ñeñeen lañu.
5,"Les gens n'étaient pas, habituellement, bavards.",Nit ñi daawuñu coow.,"Nit ñi nee nañu, Faatim la."
118,Bonne soirée.,Naka ngon si.,Jaam nga fanane.
12,Tout autre mouton que tu vois.,Meneen xar mépp moo gis.,Keneen ŋgay gis.
150,Une photo datant d'il y a soixante et neuf ans...,Nataalu juróom benn fukk ak juróom ñeent day w...,Dafa yëngu woon té beg lool ba mu daanal bunt ...
